In [9]:
import simpy
import numpy as np

# Parámetros globales
N = 5  # Número de ONUs
R = 1_000  # Tasa total de transmisión (Mbps)
T_cycle = 2  # Longitud del ciclo de polling (ms)
T_guard = 0.1  # Tiempo de resguardo (ms)
W = [0.3, 0.2, 0.1, 0.2, 0.2]  # SLA de cada ONU (debe sumar 1)

# Calcular B_min para cada ONU (constante para cada ONU)
B_min = [(T_cycle - 2 * N * T_guard) * W[i] * R / (8 * sum(W)) for i in range(N)]

# Clase ONU
class ONU:
    def __init__(self, id, avg_rate, b_min):
        self.id = id
        self.avg_rate = avg_rate  # Tasa promedio de generación (paquetes/ms)
        self.ef_request = 0  # Solicitudes para EF
        self.af_request = 0  # Solicitudes para AF
        self.be_request = 0  # Solicitudes para BE
        self.ef_grant = 0  # Asignación EF
        self.af_grant = 0  # Asignación AF
        self.be_grant = 0  # Asignación BE
        self.b_min = b_min  # B_min calculado previamente
        self.b_min_prime = b_min  # B_min' dinámico

    def generate_requests(self):
        """Generar solicitudes acumuladas para el ciclo."""
        self.ef_request = np.random.poisson(self.avg_rate * 0.2)  # 20% EF
        self.af_request = np.random.poisson(self.avg_rate * 0.5)  # 50% AF
        self.be_request = np.random.poisson(self.avg_rate * 0.3)  # 30% BE

# Clase OLT
class OLT:
    def __init__(self, env, num_onus, cycle_time, total_bandwidth, guard_time):
        self.env = env
        self.num_onus = num_onus
        self.cycle_time = cycle_time
        self.total_bandwidth = total_bandwidth
        self.guard_time = guard_time
        self.onus = []  # Lista de ONUs

    def allocate_bandwidth(self):
        """Asignar ancho de banda basado en prioridades."""
        # Paso 1: Calcular g_i^H (EF)
        for onu in self.onus:
            onu.ef_grant = onu.ef_request  # g_i^H = H_i
            onu.b_min_prime = max(0, onu.b_min - onu.ef_grant)  # Actualizar B_min'

        # Paso 2: Calcular g_i^M (AF) y g_i^L (BE)
        for onu in self.onus:
            onu.af_grant = min(onu.af_request, onu.b_min_prime)  # g_i^M
            onu.be_grant = max(0, onu.b_min_prime - onu.af_grant)  # g_i^L

        # Paso 3: Ajustar G_i^M y G_i^L según la fórmula
        for onu in self.onus:
            onu.af_grant = min(onu.af_request + onu.b_min_prime - onu.ef_grant, onu.b_min_prime)  # G_i^M
            onu.be_grant = max(0, onu.b_min - onu.ef_grant - min(onu.af_request, onu.b_min_prime))  # G_i^L

    def run(self):
        """Ejecutar ciclos de polling."""
        while True:
            # Recolectar solicitudes y asignar ancho de banda
            for onu in self.onus:
                onu.generate_requests()
            self.allocate_bandwidth()
            
            # Imprimir resultados del ciclo
            print(f"Cycle {self.env.now}:")
            for onu in self.onus:
                print(f"ONU {onu.id}: EF={onu.ef_request}/{onu.ef_grant:.2f}, "
                      f"AF={onu.af_request}/{onu.af_grant:.2f}, "
                      f"BE={onu.be_request}/{onu.be_grant:.2f}, "
                      f"B_min'={onu.b_min_prime:.2f}")
            yield self.env.timeout(self.cycle_time)

# Crear entorno
env = simpy.Environment()
olt = OLT(env, N, T_cycle, R, T_guard)

# Crear ONUs y agregarlas al OLT
for i in range(N):
    onu = ONU(i, avg_rate=100, b_min=B_min[i])  # Tasa promedio de generación (paquetes/ms)
    olt.onus.append(onu)

# Iniciar simulación
env.process(olt.run())
env.run(until=10)  # Simular 10 ms

Cycle 0:
ONU 0: EF=12/12.00, AF=47/363.00, BE=31/316.00, B_min'=363.00
ONU 1: EF=20/20.00, AF=35/230.00, BE=37/195.00, B_min'=230.00
ONU 2: EF=17/17.00, AF=57/108.00, BE=33/51.00, B_min'=108.00
ONU 3: EF=18/18.00, AF=53/232.00, BE=27/179.00, B_min'=232.00
ONU 4: EF=20/20.00, AF=54/230.00, BE=21/176.00, B_min'=230.00
Cycle 2:
ONU 0: EF=28/28.00, AF=51/347.00, BE=32/296.00, B_min'=347.00
ONU 1: EF=22/22.00, AF=45/228.00, BE=32/183.00, B_min'=228.00
ONU 2: EF=20/20.00, AF=57/105.00, BE=34/48.00, B_min'=105.00
ONU 3: EF=18/18.00, AF=54/232.00, BE=28/178.00, B_min'=232.00
ONU 4: EF=21/21.00, AF=48/229.00, BE=30/181.00, B_min'=229.00
Cycle 4:
ONU 0: EF=18/18.00, AF=42/357.00, BE=26/315.00, B_min'=357.00
ONU 1: EF=19/19.00, AF=51/231.00, BE=27/180.00, B_min'=231.00
ONU 2: EF=15/15.00, AF=48/110.00, BE=29/62.00, B_min'=110.00
ONU 3: EF=21/21.00, AF=44/229.00, BE=34/185.00, B_min'=229.00
ONU 4: EF=16/16.00, AF=56/234.00, BE=29/178.00, B_min'=234.00
Cycle 6:
ONU 0: EF=18/18.00, AF=50/357.00, BE=